# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper`.

You will find more info about virtual environments [here](../2.python/1.python_fundamentals/02.Package-Managers/00.Package_Managers.md).

Activate it.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: ` https://country-leaders.herokuapp.com/status``
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [684]:

# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.herokuapp.com"
# assign the /status endpoint to another variable called status_url (1 line)
status_url =  "https://country-leaders.herokuapp.com/status"
# query the /status endpoint using the get() method and store it in the req variable (1 line)
req_status = requests.get(status_url)
# check the status_code using a condition and print appropriate messages (4 lines)
req_status.content

b'"Alive"'

### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [629]:
# Set the country_url variable (1 line)
countries_url = 'https://country-leaders.herokuapp.com/countries'
# query the /country endpoint using the get() method and store it in the req variable (1 line)
req_country = requests.get(countries_url)
# Get the JSON content and store it in the countries variable (1 line)
countries = req_country.json()
# display the request's status code and the countries variable (1 line)
req_country.content

b'{"message":"The cookie is missing"}'

### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [630]:
# Set the cookie_url variable (1 line)
cookie_url = 'https://country-leaders.herokuapp.com/cookie'
# Query the enpoint, set the cookies variable and display it (3 lines)
req_cookies = requests.get(cookie_url)

Try to query the countries endpoint using the cookie, save the output and print it.

In [631]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
req_countries = requests.get(countries_url, cookies = req_cookies.cookies)
# and check the status code and the countries variable's content (3 lines)
countries = req_countries.json()
countries

['be', 'fr', 'us', 'ma', 'ru']

Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [653]:
cookie_url = 'https://country-leaders.herokuapp.com/cookie'
cookies = requests.get(cookie_url).cookies
cookies_req = requests.get(cookie_url).cookies
print(cookies_req)

<RequestsCookieJar[<Cookie user_cookie=8a53cd37-b18b-44a5-85cc-f29ef9e4edaa for country-leaders.herokuapp.com/>]>


In [654]:
def get_leaders():

    # 1. Urls we are going to use
    root_url = 'https://country-leaders.herokuapp.com/'
    cookie_url = 'https://country-leaders.herokuapp.com/cookie'
    leaders_url = 'https://country-leaders.herokuapp.com/leaders'
    countries_url = 'https://country-leaders.herokuapp.com/countries'

    # 2. Cookie request function
    cookies_req = requests.get(cookie_url).cookies
    print(cookies_req)

    # 3. Display the countries 
    countries_req = requests.get(countries_url, cookies = cookies_req)
    countries = countries_req.json()
    print(countries)

    # 4. Loop over countries and save leaders in a dict
    leaders_per_country = {}

    for country in countries:
        leader_req = requests.get(leaders_url, params={'country':country}, cookies=cookies_req)
        leaders_per_country[country] = leader_req.json()

    # 5. Return the dictionnay

    return leaders_per_country


In [655]:
get_leaders()

<RequestsCookieJar[<Cookie user_cookie=8efaaa5e-99cd-4172-8455-59748d6a6dd9 for country-leaders.herokuapp.com/>]>
['be', 'fr', 'us', 'ma', 'ru']


{'be': [{'id': 'Q12978',
   'first_name': 'Guy',
   'last_name': 'Verhofstadt',
   'birth_date': '1953-04-11',
   'death_date': None,
   'place_of_birth': 'Dendermonde',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt',
   'start_mandate': '1999-07-12',
   'end_mandate': '2008-03-20'},
  {'id': 'Q12981',
   'first_name': 'Yves',
   'last_name': 'Leterme',
   'birth_date': '1960-10-06',
   'death_date': None,
   'place_of_birth': 'Wervik',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme',
   'start_mandate': '2009-11-25',
   'end_mandate': '2011-12-06'},
  {'id': 'Q12983',
   'first_name': 'Herman',
   'last_name': 'Van Rompaey',
   'birth_date': '1947-10-31',
   'death_date': None,
   'place_of_birth': 'Etterbeek',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy',
   'start_mandate': '2008-12-30',
   'end_mandate': '2009-11-25'},
  {'id': 'Q14989',
   'first_name': 'Léon',
   'last_name': 'Delacroix',
   'birth_date': '1867-12-27'

In [681]:
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders_url = root_url + "/leaders"

# and check its content (4 lines)
req_leaders = requests.get(leaders_url, cookies = cookies)
leaders = req_leaders.json()
print(leaders)

{'detail': 'Not Found'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.herokuapp.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [683]:
'''Not Happy!!!'''


# Set the leaders_url variable (1 line)
root_url = "https://country-leaders.herokuapp.com"
cookiez = requests.get("https://country-leaders.herokuapp.com/cookie").cookies
print(cookiez)
leaders_url = root_url + "/leaders"
# query the /leaders endpoint using cookies and parameters (take any country in countries)
req_leaders = requests.get(leaders_url, cookies=cookies_req, params="country=fr")
# check the result (3 lines) 
print(req_leaders.json())


<RequestsCookieJar[<Cookie user_cookie=6d047b2b-9f6f-4a32-a5ee-1abf3471b51a for country-leaders.herokuapp.com/>]>
{'detail': 'Not Found'}


### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each entry in a dictionary called `leaders_per_country` like this one: 
`{'fr':[{leader0}, {leader1},...],
'en':[{leader0}, {leader1},...],
...
}`.

In [658]:
# 5 lines
leaders_per_country = {}
for country in countries:
    req_leader = requests.get(leaders_url, params={country:countries}, cookies = req_cookies.cookies)
    leaders_per_country[country] = req_leader.json()
print(leaders_per_country)


{'be': {'detail': 'Not Found'}, 'fr': {'detail': 'Not Found'}, 'us': {'detail': 'Not Found'}, 'ma': {'detail': 'Not Found'}, 'ru': {'detail': 'Not Found'}}


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [659]:
# 15 lines
def get_leaders():
    root_url = 'https://country-leaders.herokuapp.com/'
    cookie_url = 'https://country-leaders.herokuapp.com/cookie'
    leaders_url = 'https://country-leaders.herokuapp.com/leaders'
    country_url = 'https://country-leaders.herokuapp.com/country'
    cookies_req = requests.get(cookie_url)
    req_country = requests.get(country_url, cookies = cookies_req.cookies)
    countries = req_country.json()
    leaders_per_country = {}
    for country in countries:
        leader_req = requests.get(leaders_url, params={'country':country}, cookies=cookies_req.cookies)
        leaders_per_country[country] = leader_req.json()
    return leaders_per_country

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [660]:
leaders_per_country = get_leaders()
leaders_per_country

{'detail': {'message': "This country is not covered by the API. Please pick one among ['be', 'fr', 'us', 'ma', 'ru']"}}

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [661]:
# 3 lines
import requests 
macron_url = requests.get('https://fr.wikipedia.org/wiki/Emmanuel_Macron')
#macron_url.text

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function to take a look. You will start the actual parsing in the next step.

In [662]:
#3 lines
import bs4
soup = bs4.BeautifulSoup(macron_url.content, "html.parser")
print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="fr">
 <head>
  <meta charset="utf-8"/>
  <title>
   Emmanuel Macron — Wikipédia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janvier","février","mars","avril","mai","juin","juillet","août","septembre","octobre","novembre","décembre"],"wgRequestId":"d3096c99-95ac-4015-b040-7834084e1cea","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Emmanuel_Macron","wgTitle":"Emmanuel Macron","wgCurRevisionId":197054612,"wgRevisionId":197054612,"wgArticleId":6298771,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Article contenant un lien mort","Page dont la protection est à vérifier","Page en semi-protection étendue","Page au nom prot

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [663]:
# 2 lines
paragraphs = bs4.BeautifulSoup(macron_url.content, "html.parser")
#paragraphs

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [664]:
# 6 lines
def get_first_paragraph(url):
    req = requests.get(url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")
    for paragraph in soup.find_all('p'):
        if paragraph.find_all('b'):
            return paragraph.text


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [665]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)
    req = requests.get(wikipedia_url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")
    for paragraph in soup.find_all('p'):
        if paragraph.find_all('b'):
            return paragraph.text

get_first_paragraph('https://fr.wikipedia.org/wiki/Emmanuel_Macron')

https://fr.wikipedia.org/wiki/Emmanuel_Macron


"Emmanuel Macron (/ɛmanɥɛl makʁɔ̃/[e] Écouter), né le 21 décembre 1977 à Amiens (France), est un haut fonctionnaire et homme d'État français. Il est président de la République française depuis le 14 mai 2017.\n"

### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [666]:
# 4 lines
import re
text = get_first_paragraph('https://fr.wikipedia.org/wiki/Emmanuel_Macron')
regex = r".\[.\]"
paragraph_correct = re.sub(regex, "", text)
paragraph_correct


https://fr.wikipedia.org/wiki/Emmanuel_Macron


"Emmanuel Macron (/ɛmanɥɛl makʁɔ̃ Écouter), né le 21 décembre 1977 à Amiens (France), est un haut fonctionnaire et homme d'État français. Il est président de la République française depuis le 14 mai 2017.\n"

Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [667]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    req = requests.get(wikipedia_url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")
    for paragraph in soup.find_all('p'):
        if paragraph.find_all('b'):
            re.sub(r".\[.\]", ",", paragraph.text)
            text = paragraph.text
            regex = r".\[.\]"
            paragraph_correct = re.sub(regex, "", text)
            return paragraph_correct

get_first_paragraph('https://fr.wikipedia.org/wiki/Emmanuel_Macron')

"Emmanuel Macron (/ɛmanɥɛl makʁɔ̃ Écouter), né le 21 décembre 1977 à Amiens (France), est un haut fonctionnaire et homme d'État français. Il est président de la République française depuis le 14 mai 2017.\n"

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [668]:
# < 20 lines
def get_first_paragraph(wikipedia_url):
    req = requests.get(wikipedia_url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")
    for paragraph in soup.find_all('p'):
        if paragraph.find_all('b'):
            re.sub(r".\[.\]", ",", paragraph.text)
            text = paragraph.text
            regex1 = r".\[.\]"
            paragraph_correct = re.sub(regex1, "", text)
            regex2 = r"\("
            paragraph_correct = re.sub(regex2, ", ", paragraph_correct)
            regex3 = r"\)"
            paragraph_correct = re.sub(regex3, " ", paragraph_correct)
            return paragraph_correct

get_first_paragraph('https://fr.wikipedia.org/wiki/Emmanuel_Macron')

"Emmanuel Macron , /ɛmanɥɛl makʁɔ̃ Écouter , né le 21 décembre 1977 à Amiens , France , est un haut fonctionnaire et homme d'État français. Il est président de la République française depuis le 14 mai 2017.\n"

### Caching
Good job! This function will be called many times. First because there are many leaders to loop through. Second because you are developing your code and are likely to re-run your loop multiple times. Third because the API might change and contain new leaders and you might want to update your data periodically.

For all these reasons, it can be benificial to *cache* the result of the `get_first_paragraph()` function so that if it called with the same parameter, you expect the same output and thus do not need to query Wikipedia another time.

You will use the [lru_cache](https://docs.python.org/3/library/functools.html#functools.lru_cache) decorator from the `functools` module. It is a tiny code change that will speed things up big time. You can use a `maxsize` of None. If you have questions about decorators, you will find more information [here](../../2.python/2.python_advanced/07.Decorator/decorators.ipynb)

In [669]:
# < 20 lines
import functools
functools

@functools.lru_cache(maxsize=None)
def get_first_paragraph(wikipedia_url):
    req = requests.get(wikipedia_url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")
    for paragraph in soup.find_all('p'):
        if paragraph.find_all('b'):
            re.sub(r".\[.\]", ",", paragraph.text)
            text = paragraph.text
            regex1 = r".\[.\]"
            paragraph_correct = re.sub(regex1, "", text)
            regex2 = r"\("
            paragraph_correct = re.sub(regex2, ", ", paragraph_correct)
            regex3 = r"\)"
            paragraph_correct = re.sub(regex3, " ", paragraph_correct)
            return paragraph_correct

get_first_paragraph('https://fr.wikipedia.org/wiki/Emmanuel_Macron')


"Emmanuel Macron , /ɛmanɥɛl makʁɔ̃ Écouter , né le 21 décembre 1977 à Amiens , France , est un haut fonctionnaire et homme d'État français. Il est président de la République française depuis le 14 mai 2017.\n"

Let's check the speedup!

In [670]:
%%timeit
get_first_paragraph('https://fr.wikipedia.org/wiki/Emmanuel_Macron')

120 ns ± 16.1 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [671]:
##Not Happy!!!

# 19 lines

def get_leaders():
   cookies = requests.get(cookie_url).cookies
   countries = req_country.json()
   
   leaders_per_country = {}
   
   for country in countries:
       req_leaders = requests.get(leaders_url, params={'country':country}, cookies=cookies)
       leaders_per_country[country] = req_leaders.json()
       
       for leader in leaders_per_country[country]:
           leader["wikipedia_url"] = get_first_paragraph(leader["wikipedia_url"])
           leader["wikipedia_first_paragraph"] = leader.pop["wikipedia_url"]
           return leaders_per_country
           

In [672]:
# Check the output of your function (2 lines)
lead = get_leaders()
print(lead)

TypeError: string indices must be integers

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function to check if the `status_code` is a cookie error. In which case, get new ones and query the api again.

If not, you may have a fast connection or a bit of luck. Do it anyway.

In [ ]:
# 22 lines


Check the output of your function again.

In [ ]:
%%time
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [ ]:
# <20 lines

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [ ]:
# 22 lines


Test your new functions.



In [ ]:
# Just run it.
%%time
leaders_per_country = get_leaders()

Using a Session makes the first call to `get_leaders()` faster however, for technical reasons, the `@lru_cache` decorator no longer works. 

Problem: The `session` parameter is not *hashable* and since the cache is a dictionary using all the parameters as key, this poses a problem.

Solution: Create our own cache that only cares about the url.

The code for this is provided. You should only read it.

In [ ]:
cache = {}
def hashable_cache(f):
    def inner(url, session):
        if url not in cache:
            cache[url] = f(url, session)
        return cache[url]
    return inner

Overwrite the `get_first_paragraph()` function to use the new decorator like this: `@hashable_cache`.


In [ ]:
# <20 lines


Check that the cache is working again.

In [ ]:
# Just run it.
%%time
leaders_per_country = get_leaders()

## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Make a tour in our [file handling section](../../2.python/2.python_advanced/04.File-handling/file_handling.ipynb) to know more about how to read and write files using Python.

In [ ]:
# 3 lines


Make sure the file can be read back. Write the code to read the file.

In [ ]:
#3 lines

Make a function `save()` to call this code easily.

In [ ]:
# 3 lines


In [ ]:
# Call the function (1 line)

## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!